# Machine Translation

### Dataset Preparation:

In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [2]:
file = open("nya.tsv", 'r', encoding = "utf8")
raw_data = []

for line in file:
    pos = line.find("CC-BY")
    line = line[:pos-1]
    
    # Split the data into english and Italian
    eng, nya = line.split('\t')
    
    # form tuples of the data
    data = eng, nya
    raw_data.append(data)
    
file.close()

def convert(list): 
    return tuple(list) 
  
data = convert(raw_data)

In [3]:
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(s):
    s = unicode_to_ascii(s.lower())
    s = re.sub(r'([!.?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    s = re.sub(r'\s+', r' ', s)

    s = s.strip()
    s = '<start>' +' '+ s +' '+' <end>'
    return s

In [4]:
# Limiting the data and Splitting into seperate lists and add tokens

data = data[:27000]

lang_eng = []
lang_nya = []

raw_data_en, raw_data_nya = list(zip(*data))
raw_data_en, raw_data_nya = list(raw_data_en), list(raw_data_nya)

for i, j in zip(raw_data_en, raw_data_nya):
  preprocessed_data_en = preprocess_sentence(i)
  preprocessed_data_nya = preprocess_sentence(j)
  lang_eng.append(preprocessed_data_en)
  lang_nya.append(preprocessed_data_nya)

def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

input_tensor, inp_lang = tokenize(lang_nya)
target_tensor, targ_lang = tokenize(lang_eng)

max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [5]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])



162 162 41 41
Input Language; index to word mapping
1 ----> <start>
4 ----> to
77 ----> agree
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
31 ----> bvomera
3 ----> .
2 ----> <end>


In [6]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE

vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset

Metal device set to: Apple M1


2022-11-19 14:03:59.468543: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-19 14:03:59.468979: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<BatchDataset element_spec=(TensorSpec(shape=(64, 12), dtype=tf.int32, name=None), TensorSpec(shape=(64, 5), dtype=tf.int32, name=None))>

### Encoder Architecture:

In [7]:
class Encoder(tf.keras.Model):

    def __init__(self, inp_vocab_size, embedding_size, lstm_size, input_length):
        super(Encoder, self).__init__()
        
        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        
        self.lstm_size = lstm_size
        self.embedding = tf.keras.layers.Embedding(inp_vocab_size, embedding_size)
        self.lstm = tf.keras.layers.LSTM(lstm_size, return_sequences=True, return_state=True)

    def call(self, input_sequence, states):
      
        embed = self.embedding(input_sequence)
        output, state_h, state_c = self.lstm(embed, initial_state=states)

        return output, state_h, state_c
    
    def initialize_states(self,batch_size):
    
        return (tf.zeros([batch_size, self.lstm_size]),
                tf.zeros([batch_size, self.lstm_size]))

### Dot Attention:

In [8]:
class Attention(tf.keras.layers.Layer):
    def __init__(self,scoring_function, att_units):
        super(Attention, self).__init__()
        
        self.scoring_function = scoring_function
        self.att_units = att_units

        if self.scoring_function=='dot':
            pass
            # For general, it would be self.wa = tf.keras.layers.Dense(att_units)


    def call(self,decoder_hidden_state,encoder_output):

        if self.scoring_function == 'dot':
            
            new_state = tf.expand_dims(decoder_hidden_state, -1)
            score = tf.matmul(encoder_output, new_state)
            weights = tf.nn.softmax(score, axis=1)
            context = weights * encoder_output
            context_vector = tf.reduce_sum(context, axis=1)
                                
            return context_vector, weights

### One Step Decoder

In [9]:
class One_Step_Decoder(tf.keras.Model):
    def __init__(self, tar_vocab_size, embedding_dim, input_length, dec_units, score_fun, att_units):
        super(One_Step_Decoder, self).__init__()
        # Initialize decoder embedding layer, LSTM and any other objects needed
        self.tar_vocab_size = tar_vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units
        self.embedding = tf.keras.layers.Embedding(self.tar_vocab_size, self.embedding_dim, 
                                                   input_length=self.input_length)
        
        self.lstm = tf.keras.layers.LSTM(self.dec_units, return_sequences=True, 
                                         return_state=True)
        
        self.output_layer = tf.keras.layers.Dense(self.tar_vocab_size)
        
        self.attention = Attention(self.score_fun, self.att_units)

    def call(self, input_to_decoder, encoder_output, state_h, state_c):
        
        result = self.embedding(input_to_decoder)
        
        context_vector, weights = self.attention(state_h, encoder_output)
        
        concat = tf.concat([tf.expand_dims(context_vector, 1), result], axis=-1)
        
        decoder_output, hidden_state, cell_state = self.lstm(concat, initial_state=[state_h, state_c])
        
        final_output = tf.reshape(decoder_output, (-1, decoder_output.shape[2]))
        final_output = self.output_layer(final_output)
        
        return final_output, hidden_state, cell_state, weights, context_vector

### Decoder

In [10]:
class Decoder(tf.keras.Model):
    def __init__(self, out_vocab_size, embedding_dim, output_length, dec_units ,score_fun ,att_units):
        #Intialize necessary variables and create an object from the class onestepdecoder
        super(Decoder, self).__init__()
        self.out_vocab_size = out_vocab_size
        self.embedding_dim = embedding_dim
        self.output_length = output_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units
        self.onestepdecoder = One_Step_Decoder(self.out_vocab_size, self.embedding_dim, self.output_length,
                                               self.dec_units, self.score_fun, self.att_units)
        
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state):
        
        all_outputs= tf.TensorArray(tf.float32, size=input_to_decoder.shape[1], name="output_arrays")
        
        
        for timestep in range(input_to_decoder.shape[1]):
            output, decoder_hidden_state, decoder_cell_state, weights, context_vector = self.onestepdecoder(
                                                                                    input_to_decoder[:,timestep:timestep+1], 
                                                                                    encoder_output, 
                                                                                    decoder_hidden_state,
                                                                                    decoder_cell_state)
            
            all_outputs = all_outputs.write(timestep, output)
        
        all_outputs = tf.transpose(all_outputs.stack(), (1, 0, 2)) 

        return all_outputs

### Call The Encoder Decoder Architecture:

In [11]:
class encoder_decoder(tf.keras.Model):
    def __init__(self, inp_vocab_size, out_vocab_size, embedding_size, lstm_size, 
                 input_length, output_length, dec_units ,score_fun ,att_units, batch_size):
        
        super(encoder_decoder, self).__init__()
        
        self.encoder = Encoder(inp_vocab_size, embedding_size, lstm_size, input_length)
        self.decoder = Decoder(out_vocab_size, embedding_size, output_length, 
                               dec_units, score_fun, att_units)
    
    def call(self, data):
        
        input_sequence, input_to_decoder = data[0],data[1]
        initial_state = self.encoder.initialize_states(batch_size=64)
        encoder_output, state_h, state_c = self.encoder(input_sequence, initial_state)
        decoder_hidden_state = state_h
        decoder_cell_state = state_c
        decoder_output = self.decoder(input_to_decoder, encoder_output, decoder_hidden_state, decoder_cell_state)
        
        return decoder_output

### Custom Loss Function: 

In [12]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

optimizer = tf.keras.optimizers.Adam()

### Training:

In [13]:
!mkdir logs

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("dot.h5", monitor='val_loss', verbose=1, save_weights_only=True)

logdir='logs'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

input_vocab_size = len(inp_lang.word_index)+1
output_vocab_size = len(targ_lang.word_index)+1

input_len = max_length_inp
output_len = max_length_targ

lstm_size = 128
att_units = 256
dec_units = 128
embedding_size = 300
embedding_dim = 300
score_fun = 'dot'
steps = len(input_tensor)//64
batch_size=64

model = encoder_decoder(input_vocab_size,output_vocab_size,embedding_size,lstm_size,input_len,output_len,dec_units,score_fun,att_units, batch_size)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=model.layers[0],
                                 decoder=model.layers[1])

mkdir: logs: File exists


In [14]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden,enc_state = model.layers[0](inp, enc_hidden)


    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    for t in range(1, targ.shape[1]):
      predictions = model.layers[1](dec_input,enc_output,enc_hidden,enc_state)

      loss += loss_function(targ[:, t], predictions)

      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = model.layers[0].trainable_variables + model.layers[1].trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [16]:
EPOCHS = 500 # specifying the number of epochs or runs for training the model

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = model.layers[0].initialize_states(64)
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
      
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.0029
Epoch 1 Loss 0.0071
Time taken for 1 epoch 0.23241329193115234 sec

Epoch 2 Batch 0 Loss 0.0055
Epoch 2 Loss 0.0071
Time taken for 1 epoch 0.16886186599731445 sec

Epoch 3 Batch 0 Loss 0.0033
Epoch 3 Loss 0.0084
Time taken for 1 epoch 0.09501528739929199 sec

Epoch 4 Batch 0 Loss 0.0032
Epoch 4 Loss 0.0067
Time taken for 1 epoch 0.16767191886901855 sec

Epoch 5 Batch 0 Loss 0.0118
Epoch 5 Loss 0.0084
Time taken for 1 epoch 0.08693408966064453 sec

Epoch 6 Batch 0 Loss 0.0110
Epoch 6 Loss 0.0071
Time taken for 1 epoch 0.18056797981262207 sec

Epoch 7 Batch 0 Loss 0.0083
Epoch 7 Loss 0.0079
Time taken for 1 epoch 0.08929896354675293 sec

Epoch 8 Batch 0 Loss 0.0096
Epoch 8 Loss 0.0080
Time taken for 1 epoch 0.1609790325164795 sec

Epoch 9 Batch 0 Loss 0.0048
Epoch 9 Loss 0.0075
Time taken for 1 epoch 0.08850502967834473 sec

Epoch 10 Batch 0 Loss 0.0024
Epoch 10 Loss 0.0062
Time taken for 1 epoch 0.16075897216796875 sec

Epoch 11 Batch 0 Loss 0.0045
Epoch 11 L

Epoch 86 Loss 0.0082
Time taken for 1 epoch 0.15804219245910645 sec

Epoch 87 Batch 0 Loss 0.0063
Epoch 87 Loss 0.0092
Time taken for 1 epoch 0.0880889892578125 sec

Epoch 88 Batch 0 Loss 0.0071
Epoch 88 Loss 0.0081
Time taken for 1 epoch 0.16056180000305176 sec

Epoch 89 Batch 0 Loss 0.0028
Epoch 89 Loss 0.0067
Time taken for 1 epoch 0.08514595031738281 sec

Epoch 90 Batch 0 Loss 0.0089
Epoch 90 Loss 0.0059
Time taken for 1 epoch 0.15961503982543945 sec

Epoch 91 Batch 0 Loss 0.0047
Epoch 91 Loss 0.0071
Time taken for 1 epoch 0.08791708946228027 sec

Epoch 92 Batch 0 Loss 0.0044
Epoch 92 Loss 0.0048
Time taken for 1 epoch 0.1611461639404297 sec

Epoch 93 Batch 0 Loss 0.0097
Epoch 93 Loss 0.0070
Time taken for 1 epoch 0.08649778366088867 sec

Epoch 94 Batch 0 Loss 0.0097
Epoch 94 Loss 0.0060
Time taken for 1 epoch 0.1599578857421875 sec

Epoch 95 Batch 0 Loss 0.0024
Epoch 95 Loss 0.0070
Time taken for 1 epoch 0.08831524848937988 sec

Epoch 96 Batch 0 Loss 0.0024
Epoch 96 Loss 0.0081
Ti

Epoch 170 Loss 0.0080
Time taken for 1 epoch 0.15845894813537598 sec

Epoch 171 Batch 0 Loss 0.0025
Epoch 171 Loss 0.0037
Time taken for 1 epoch 0.08760190010070801 sec

Epoch 172 Batch 0 Loss 0.0026
Epoch 172 Loss 0.0058
Time taken for 1 epoch 0.15862703323364258 sec

Epoch 173 Batch 0 Loss 0.0046
Epoch 173 Loss 0.0080
Time taken for 1 epoch 0.08713173866271973 sec

Epoch 174 Batch 0 Loss 0.0046
Epoch 174 Loss 0.0060
Time taken for 1 epoch 0.16225624084472656 sec

Epoch 175 Batch 0 Loss 0.0111
Epoch 175 Loss 0.0091
Time taken for 1 epoch 0.08931708335876465 sec

Epoch 176 Batch 0 Loss 0.0023
Epoch 176 Loss 0.0090
Time taken for 1 epoch 0.16410112380981445 sec

Epoch 177 Batch 0 Loss 0.0064
Epoch 177 Loss 0.0079
Time taken for 1 epoch 0.0853111743927002 sec

Epoch 178 Batch 0 Loss 0.0118
Epoch 178 Loss 0.0081
Time taken for 1 epoch 0.1591031551361084 sec

Epoch 179 Batch 0 Loss 0.0090
Epoch 179 Loss 0.0068
Time taken for 1 epoch 0.08842992782592773 sec

Epoch 180 Batch 0 Loss 0.0110
Ep

Epoch 254 Loss 0.0053
Time taken for 1 epoch 0.1746528148651123 sec

Epoch 255 Batch 0 Loss 0.0058
Epoch 255 Loss 0.0080
Time taken for 1 epoch 0.08748698234558105 sec

Epoch 256 Batch 0 Loss 0.0003
Epoch 256 Loss 0.0037
Time taken for 1 epoch 0.16765499114990234 sec

Epoch 257 Batch 0 Loss 0.0140
Epoch 257 Loss 0.0081
Time taken for 1 epoch 0.10105180740356445 sec

Epoch 258 Batch 0 Loss 0.0142
Epoch 258 Loss 0.0091
Time taken for 1 epoch 0.16910314559936523 sec

Epoch 259 Batch 0 Loss 0.0081
Epoch 259 Loss 0.0056
Time taken for 1 epoch 0.08582806587219238 sec

Epoch 260 Batch 0 Loss 0.0072
Epoch 260 Loss 0.0083
Time taken for 1 epoch 0.16904997825622559 sec

Epoch 261 Batch 0 Loss 0.0053
Epoch 261 Loss 0.0046
Time taken for 1 epoch 0.09167194366455078 sec

Epoch 262 Batch 0 Loss 0.0042
Epoch 262 Loss 0.0091
Time taken for 1 epoch 0.17919301986694336 sec

Epoch 263 Batch 0 Loss 0.0076
Epoch 263 Loss 0.0081
Time taken for 1 epoch 0.09144306182861328 sec

Epoch 264 Batch 0 Loss 0.0067
E

Epoch 338 Loss 0.0081
Time taken for 1 epoch 0.16122126579284668 sec

Epoch 339 Batch 0 Loss 0.0072
Epoch 339 Loss 0.0080
Time taken for 1 epoch 0.08742713928222656 sec

Epoch 340 Batch 0 Loss 0.0067
Epoch 340 Loss 0.0078
Time taken for 1 epoch 0.15874004364013672 sec

Epoch 341 Batch 0 Loss 0.0065
Epoch 341 Loss 0.0067
Time taken for 1 epoch 0.08835005760192871 sec

Epoch 342 Batch 0 Loss 0.0022
Epoch 342 Loss 0.0066
Time taken for 1 epoch 0.16133975982666016 sec

Epoch 343 Batch 0 Loss 0.0110
Epoch 343 Loss 0.0078
Time taken for 1 epoch 0.08833694458007812 sec

Epoch 344 Batch 0 Loss 0.0109
Epoch 344 Loss 0.0069
Time taken for 1 epoch 0.1609022617340088 sec

Epoch 345 Batch 0 Loss 0.0135
Epoch 345 Loss 0.0069
Time taken for 1 epoch 0.08762335777282715 sec

Epoch 346 Batch 0 Loss 0.0064
Epoch 346 Loss 0.0057
Time taken for 1 epoch 0.15897226333618164 sec

Epoch 347 Batch 0 Loss 0.0132
Epoch 347 Loss 0.0090
Time taken for 1 epoch 0.08631110191345215 sec

Epoch 348 Batch 0 Loss 0.0047
E

Epoch 422 Loss 0.0063
Time taken for 1 epoch 0.16673612594604492 sec

Epoch 423 Batch 0 Loss 0.0091
Epoch 423 Loss 0.0056
Time taken for 1 epoch 0.08894991874694824 sec

Epoch 424 Batch 0 Loss 0.0098
Epoch 424 Loss 0.0080
Time taken for 1 epoch 0.16266226768493652 sec

Epoch 425 Batch 0 Loss 0.0129
Epoch 425 Loss 0.0066
Time taken for 1 epoch 0.08823704719543457 sec

Epoch 426 Batch 0 Loss 0.0078
Epoch 426 Loss 0.0062
Time taken for 1 epoch 0.16590499877929688 sec

Epoch 427 Batch 0 Loss 0.0072
Epoch 427 Loss 0.0075
Time taken for 1 epoch 0.08843016624450684 sec

Epoch 428 Batch 0 Loss 0.0040
Epoch 428 Loss 0.0052
Time taken for 1 epoch 0.16344118118286133 sec

Epoch 429 Batch 0 Loss 0.0027
Epoch 429 Loss 0.0081
Time taken for 1 epoch 0.08883118629455566 sec

Epoch 430 Batch 0 Loss 0.0054
Epoch 430 Loss 0.0079
Time taken for 1 epoch 0.16922712326049805 sec

Epoch 431 Batch 0 Loss 0.0045
Epoch 431 Loss 0.0065
Time taken for 1 epoch 0.08499288558959961 sec

Epoch 432 Batch 0 Loss 0.0045


In [ ]:
# !pip install -q tensorflow-recommenders
# !pip install --upgrade pip

In [ ]:
# Saving the keras model
import tensorflow_recommenders as tfrs
model.retrieval_task = tfrs.tasks.Retrieval()  # Removes the metrics.
model.compile()
tf.saved_model.save(model, '/Users/cm15/Desktop')

# Loading the Keras Model
# model = tf.saved_model.load(path_to_dir)

### Translate:

In [17]:
def predict(input_sentence):

  attention_plot = np.zeros((output_len, input_len))

  input_sentence = preprocess_sentence(input_sentence)

  inputs = [inp_lang.word_index[i] for i in input_sentence.split()]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=input_len,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''
  
  encoder_output,state_h,state_c = model.layers[0](inputs,[tf.zeros((1, lstm_size)),tf.zeros((1, lstm_size))])

  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(output_len):
   predictions,state_h,state_c,attention_weights,context_vector = model.layers[1].onestepdecoder(dec_input,
                                                                                                 encoder_output,
                                                                                                 state_h,
                                                                                                 state_c)

   attention_weights = tf.reshape(attention_weights, (-1, ))
   attention_plot[t] = attention_weights.numpy()

   predicted_id = tf.argmax(predictions[0]).numpy()

   result += targ_lang.index_word[predicted_id] + ' '

   if targ_lang.index_word[predicted_id] == '<end>':
     return result, input_sentence, attention_plot

   dec_input = tf.expand_dims([predicted_id], 0)

  return result, input_sentence, attention_plot

In [18]:
def translate(sentence):
  result, sent, attention_plot = predict(sentence)

  print('Input: %s' % (sent))
  print('Predicted translation: {}'.format(result))

In [23]:
translate("to agree")

Input: <start> to agree  <end>
Predicted translation: bvomera . <end> 


In [24]:
import gradio as gd

In [25]:
def translate(sentence):
  result, sent, attention_plot = predict(sentence)

  return result
UI = gd.Interface(translate, inputs='text', outputs='text')

In [26]:
UI.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x17749ee90>, 'http://127.0.0.1:7860/', None)